In [1]:
#Creates a custom playlist from your library where you can specify the BPM (beats per minute) to
#fit your run.  The constant BPM option will contain songs with similar BPM's that reflect the given
#peak intensity, whereas selecting "no" to constant BPM will create a playlist that gradually increases
#to peak BPM and falls for the cooldown

#When prompted for "code after redirect URI," a google.com popup will appear.
#In the URL, there will be a long string of characters after "code="
#Copy and paste the entire string from the first character after the "="
#to the end of the URL

In [2]:
import requests
import json
import base64
import datetime
from urllib.parse import urlencode
import webbrowser
import pandas as pd
import math

In [3]:
class SpotifyAPI(object):
    client_id = None
    client_secret = None
    redirect = 'http://google.com/'
    client_creds_b64 = None
    
    def __init__(self, client_id, client_secret, user_id):
        self.client_id = client_id
        self.client_secret = client_secret
        self.user_id = user_id
        
        #formats client and secret client in b64 to use for authentication
        client_creds = f"{client_id}:{client_secret}"
        self.client_creds_b64 = base64.b64encode(client_creds.encode())
        self.client_creds_b64 = self.client_creds_b64.decode()
    
    #requires: scope found in the spotify API documentation
    #modifies:
    #effects: returns the access token
    def authorize_account(self, scope):
        #opens a link to log in and get the code for authentication
        url = 'https://accounts.spotify.com/authorize'
        data = urlencode({"client_id": self.client_id, 'response_type': 'code',  "redirect_uri": self.redirect,
                        "scope": scope})
        full_url = f"{url}?{data}"
        webbrowser.open(full_url)
        
        #input code from user (hopefully will be able to automate this)
        codeURL = input("Input code found after the redirect URI: ")
        token_url = 'https://accounts.spotify.com/api/token'
        headers = {
            "Authorization": f"Basic {self.client_creds_b64}"
        }

        token_data = {"grant_type": "authorization_code",
                     "code": codeURL,
                     "redirect_uri": self.redirect}
        
        r2 = requests.post(token_url, data = token_data, headers = headers)
        
        return r2
    
    #effects: returns access token that modifies playlists
    def authorize_playlist(self):
        return self.authorize_account('playlist-modify-public')
    
    #effects: returns access token to look at user library
    def authorize_library(self):
        return self.authorize_account('user-library-read')
    
    #effects: returns an access token that can't modify account in any way
    def basic_authorization(self):
        token_url = 'https://accounts.spotify.com/api/token'
        token_data = {
                    "grant_type": "client_credentials"
                }

        token_header = {
                    "Authorization": f"Basic {self.client_creds_b64}"
                }
        r2 = requests.post(token_url, data = token_data, headers = token_header)
        return r2
    
    #requires: a request object with a library access token
    #effects: returns a dictionary of liked songs
    def liked_songs(self, r2):
        
        #populates a dictionary of liked songs
        tempoDict = {}
        numSongsInLib = input("Number of songs in library: ")
        numSongsInLib = int(numSongsInLib)
        offset = 0
        limit = 50

        while(len(tempoDict) < numSongsInLib):
            query = "https://api.spotify.com/v1/users/{}/tracks".format(
                user_id)

            data = urlencode({"limit": limit, 'offset': offset})

            response = requests.get(
                f"{query}?{data}",
                headers={
                    "Content-Type": "application/json",
                    "Authorization": "Bearer {}".format(r2.json()['access_token'])
                }
            )
            tracks = response.json()

            for i in range(0, limit):
                if(len(tempoDict) == numSongsInLib):
                    break

                tempoDict[tracks['items'][i]['track']['id']] = 0

            offset += 50
        
        return tempoDict
    
    #requires: a dictionary with song IDs and a request object with a regular access token
    #effects: returns a populated dictionary with tempos
    def tempo_dictionary(self, tempoDict, r2):
        
        for i in tempoDict:
            query = "https://api.spotify.com/v1/audio-features/{}/".format(str(i))
            response = requests.get(
                query,
                headers={
                    "Authorization": "Bearer {}".format(r2.json()['access_token'])
                }
            )
            tempoDict[i] = response.json()['tempo']
            
        return tempoDict
    
    #requires a dictionary with tempos in it
    def segment_songs_by_tempo(self, tempoDict):
        
        tempoDF = pd.DataFrame(tempoDict.items(), columns = ['songID', 'tempo'])
        tempoDF = tempoDF.set_index(['songID'])
        
        df140 = tempoDF[(tempoDF['tempo'] > 140) & (tempoDF['tempo'] < 145)]
        df145 = tempoDF[(tempoDF['tempo'] > 145) & (tempoDF['tempo'] < 150)]
        df150 = tempoDF[(tempoDF['tempo'] > 150) & (tempoDF['tempo'] < 155)]
        df155 = tempoDF[(tempoDF['tempo'] > 155) & (tempoDF['tempo'] < 160)]
        df160 = tempoDF[(tempoDF['tempo'] > 160) & (tempoDF['tempo'] < 165)]
        df165 = tempoDF[(tempoDF['tempo'] > 165) & (tempoDF['tempo'] < 170)]
        df170 = tempoDF[(tempoDF['tempo'] > 170) & (tempoDF['tempo'] < 175)]
        df175 = tempoDF[(tempoDF['tempo'] > 175) & (tempoDF['tempo'] < 180)]
        df180 = tempoDF[(tempoDF['tempo'] > 180) & (tempoDF['tempo'] < 185)]
        
        dfList = [df140, df145, df150, df155, df160, df165, df170, df175, df180]
        
        return dfList
    
    #requires: list of dataframes that are segmented by tempo
    def select_songs_by_tempo(self, dfList):
        
        lenRun = input("length of run (in minutes): ")
        lenRun = int(lenRun)
        intensity = input("Peak intensity of run (scale from 1 to 8): ")
        intensity = int(intensity)
        crestBool = input("Constant BPM (y/n)?: ")
        numSongs = math.ceil(lenRun / 3.5)
        
        bagOfSongsAsc = []
        if(crestBool == 'n'):
            listInd = 0
            for i in range(0, numSongs):
                #makes sure array is inbounds
                if(listInd > 8):
                    listInd = 8
                if(listInd < 0):
                    listInd = 0
                #increments/decrements which intensity bucket the next song will be from    
                if (i < (numSongs / 2)):
                    bagOfSongsAsc.append(dfList[round(listInd)].sample())
                    listInd += (intensity - 1)/ (((numSongs - 3) / 2) + 1)
                else:
                    bagOfSongsAsc.append(dfList[math.ceil(listInd)].sample())
                    listInd -= (intensity - 1)/ (((numSongs - 3) / 2) + 1)

        else:
            for i in range(0, numSongs):
                bagOfSongsAsc.append(dfList[intensity - 1].sample())
                
        #takes only the song ID's
        bagOfID = []
        for i in bagOfSongsAsc:
            bagOfID.append(i.index.tolist()[0])
        #concatenates a string to make ID's to URI's    
        tempBagOfID = []
        for i in bagOfID:
            i = str(i)
            temp = 'spotify:track:' + i
            tempBagOfID.append(temp)
        
        return tempBagOfID
    
    #requires: a request with a modifies playlist access token
    #effects: makes a new playlist on spotify and returns the playlist ID
    def make_new_playlist(self, r2):
        playlistName = input("Playlist Name: ")

        request_body = json.dumps({
            "name": playlistName,
            "description": "Running BPM",
            "public": True
        })

        query = "https://api.spotify.com/v1/users/{}/playlists".format(
            user_id)
        response = requests.post(
            query,
            data=request_body,
            headers={
                "Content-Type": "application/json",
                "Authorization": "Bearer {}".format(r2.json()['access_token'])
            }
        )

        return response.json()['id']
    
    #requires: playlist ID, list of songs to be added, and an access token that can modify playlists
    def populate_playlist(self, playlistID, bagOfID, r2):
        
        request_body = json.dumps({
            "uris": bagOfID
        })

        query = "https://api.spotify.com/v1/playlists/{}/tracks".format(
            playlistID)
        response = requests.post(
            query,
            data=request_body,
            headers={
                "Content-Type": "application/json",
                "Authorization": "Bearer {}".format(r2.json()['access_token'])
            }
        )
        return
    
    def tempo_main(self):
        lib_token = self.authorize_library()
        client_token = self.basic_authorization()
        playlist_token = self.authorize_playlist()
        
        songs_list = self.select_songs_by_tempo(self.segment_songs_by_tempo(\
            self.tempo_dictionary(\
                self.liked_songs(lib_token), client_token)))
        
        playlist_id = self.make_new_playlist(playlist_token)
        self.populate_playlist(playlist_id, songs_list, playlist_token)
        
        return
        

In [4]:
clientID = '92b5a2e1c39246a685c140ba257f19f5'
client_secret = 'd066cb0c77c440f5a6e270d8e19cde22'
user_id = input("Input spotify userID: ")

Input spotify userID: jackmu4


In [5]:
spot = SpotifyAPI(clientID, client_secret, user_id)

In [ ]:
spot.tempo_main()

Input code found after the redirect URI: AQBSTGFZAZowxcboH35Iq65xLaZ7PhHEBk1CFPuFRI26pBP-Yvw9MC-P1Q1IR1Ntx-GPBzoQiAwRc-jjsW6r92pmpGo7AKitkGcNzjuR9xEvbZcFQtJidJG4pLyV7z2ZDC4BKV2rksUig7g-Vo1wAhYh4XPY9naJ69VEQKLP_osD9OMId7GBx7o
Input code found after the redirect URI: AQDnYVmFDzYfljaW3u9aV4GqMvoyzDmMe6YCrKBkIvZ8ans6hJ3r3OMbBhoQtys5xIaESwGnBCCKX_w-jbXLQHgn9rMyF5zo6hAloCvdcvbkl95FeEMpNO_Cv0UI6z6pvrnyDJJT4PMMFTZBOH_uGzgpOlHTPNauziTVxh7hIbHhtfCQva-eUTwh9nk90g
Number of songs in library: 1845


# Testing Site

In [14]:
numSongs = 10
intensity = 3
listInd = 0.0

for i in range(0, numSongs):
    #makes sure array is inbounds
    if(listInd > 8):
        listInd = 8.0
    if(listInd < 0):
        listInd = 0.0
    #increments/decrements which intensity bucket the next song will be from    
    if (i < (numSongs / 2)):
        print(round(listInd), ' + ', (intensity - 1) / (((numSongs - 3) / 2) + 1))
        listInd += (intensity - 1)/ (((numSongs - 3) / 2) + 1)
    else:
        print(math.ceil(listInd), ' - ', (intensity - 1) / (((numSongs - 3) / 2) + 1))
        listInd -= (intensity - 1) / (((numSongs - 3) / 2) + 1)

0  +  0.4444444444444444
0  +  0.4444444444444444
1  +  0.4444444444444444
1  +  0.4444444444444444
2  +  0.4444444444444444
3  -  0.4444444444444444
2  -  0.4444444444444444
2  -  0.4444444444444444
1  -  0.4444444444444444
1  -  0.4444444444444444


In [21]:
lib_token = spot.authorize_library()

Input code found after the redirect URI: AQBHJ7gt9-4hY_oP9TdAkWYi89EBXknXPYVpo2K4JNVxPotCxt6Dy1dxiEV5Vdtkq5BaZ9l61HERrvxPtquASorf3I0cWo0k8Pk5f4u5-Ez_AZe2zQpUeIo5EbAeDrC34fg-dcq4Coj1hoNEaY9BGwiLL82_Q-xaw6Um-dlg3MGVnBWf3WT02Xo


In [45]:
limit = 50
offset = 150
query = "https://api.spotify.com/v1/users/{}/tracks".format(
                user_id)

data = urlencode({"limit": limit, 'offset': offset})

response = requests.get(
    f"{query}?{data}",
    headers={
        "Content-Type": "application/json",
        "Authorization": "Bearer {}".format(lib_token.json()['access_token'])
    }
)
tracks = response.json()

tracks

{'href': 'https://api.spotify.com/v1/me/tracks?offset=150&limit=50',
 'items': [{'added_at': '2020-03-12T16:52:41Z',
   'track': {'album': {'album_type': 'compilation',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0Jeckitay8SbvKwqAzWuYH'},
       'href': 'https://api.spotify.com/v1/artists/0Jeckitay8SbvKwqAzWuYH',
       'id': '0Jeckitay8SbvKwqAzWuYH',
       'name': 'Big Mountain',
       'type': 'artist',
       'uri': 'spotify:artist:0Jeckitay8SbvKwqAzWuYH'}],
     'available_markets': ['AD',
      'AE',
      'AL',
      'AR',
      'AT',
      'AU',
      'BA',
      'BE',
      'BG',
      'BH',
      'BO',
      'BR',
      'BY',
      'CA',
      'CH',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DE',
      'DK',
      'DO',
      'DZ',
      'EC',
      'EE',
      'EG',
      'ES',
      'FI',
      'FR',
      'GB',
      'GR',
      'GT',
      'HK',
      'HN',
      'HR',
      'HU',
      'ID',
      'IE',
      'I

In [46]:
tracks['items'][0]['track']

{'album': {'album_type': 'compilation',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0Jeckitay8SbvKwqAzWuYH'},
    'href': 'https://api.spotify.com/v1/artists/0Jeckitay8SbvKwqAzWuYH',
    'id': '0Jeckitay8SbvKwqAzWuYH',
    'name': 'Big Mountain',
    'type': 'artist',
    'uri': 'spotify:artist:0Jeckitay8SbvKwqAzWuYH'}],
  'available_markets': ['AD',
   'AE',
   'AL',
   'AR',
   'AT',
   'AU',
   'BA',
   'BE',
   'BG',
   'BH',
   'BO',
   'BR',
   'BY',
   'CA',
   'CH',
   'CL',
   'CO',
   'CR',
   'CY',
   'CZ',
   'DE',
   'DK',
   'DO',
   'DZ',
   'EC',
   'EE',
   'EG',
   'ES',
   'FI',
   'FR',
   'GB',
   'GR',
   'GT',
   'HK',
   'HN',
   'HR',
   'HU',
   'ID',
   'IE',
   'IL',
   'IN',
   'IS',
   'IT',
   'JO',
   'JP',
   'KW',
   'KZ',
   'LB',
   'LI',
   'LT',
   'LU',
   'LV',
   'MA',
   'MC',
   'MD',
   'ME',
   'MK',
   'MT',
   'MX',
   'MY',
   'NI',
   'NL',
   'NO',
   'NZ',
   'OM',
   'PA',
   'PE',
   'PH',
   'PL',
  

In [30]:
liked_songs = spot.liked_songs(lib_token)

Number of songs in library: 1414


In [51]:
query = "https://api.spotify.com/v1/audio-features/{}/".format('1Rvl8qsKJurfFTyWLBI9ib')
response = requests.get(
    query,
    headers={
        "Authorization": "Bearer {}".format(lib_token.json()['access_token'])
    }
)
response.json()

{'danceability': 0.651,
 'energy': 0.747,
 'key': 1,
 'loudness': -7.269,
 'mode': 0,
 'speechiness': 0.0699,
 'acousticness': 0.0528,
 'instrumentalness': 0,
 'liveness': 0.207,
 'valence': 0.776,
 'tempo': 147.518,
 'type': 'audio_features',
 'id': '1Rvl8qsKJurfFTyWLBI9ib',
 'uri': 'spotify:track:1Rvl8qsKJurfFTyWLBI9ib',
 'track_href': 'https://api.spotify.com/v1/tracks/1Rvl8qsKJurfFTyWLBI9ib',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1Rvl8qsKJurfFTyWLBI9ib',
 'duration_ms': 249520,
 'time_signature': 4}

In [ ]:
# def select_songs_by_tempo(self, dfList):
        
#     lenRun = input("length of run (in minutes): ")
#     lenRun = int(lenRun)
#     intensity = input("Peak intensity of run (scale from 1 to 10): ")
#     intensity = int(intensity) - 1
#     crestBool = input("Constant BPM (y/n)?: ")
#     numSongs = math.ceil(lenRun / 3.5)

#     bagOfSongsAsc = []
#     if(crestBool == 'n'):
#         listInd = 0
#         for i in range(0, numSongs):
#             #makes sure array is inbounds
#             if(listInd > 8):
#                 listInd = 8
#             if(listInd < 0):
#                 listInd = 0
#             #increments/decrements which intensity bucket the next song will be from    
#             if (i < (numSongs / 2) - 1):
#                 bagOfSongsAsc.append(dfList[listInd].sample())
#                 listInd += math.ceil(intensity / math.floor(numSongs / 2))
#             else:
#                 bagOfSongsAsc.append(dfList[listInd].sample())
#                 listInd -= math.ceil(intensity / math.floor(numSongs / 2))
#     else:
#         for i in range(0, numSongs):
#             bagOfSongsAsc.append(dfList[intensity - 1].sample())
#     #takes only the song ID's
#     bagOfID = []
#     for i in bagOfSongsAsc:
#         bagOfID.append(i.index.tolist()[0])
#     #concatenates a string to make ID's to URI's    
#     tempBagOfID = []
#     for i in bagOfID:
#         i = str(i)
#         temp = 'spotify:track:' + i
#         tempBagOfID.append(temp)

#     return tempBagOfID